In [ ]:
!nvidia-smi -L

In [ ]:
# Mount drive and cd to notebook folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd "/content/drive/MyDrive/Thesis/rhythmic-relationships/scripts"

import sys
COLAB_WORKDIR = "/content/drive/MyDrive/Thesis/rhythmic-relationships/scripts"
if COLAB_WORKDIR not in sys.path:
  sys.path.append(COLAB_WORKDIR)
print(sys.path)

In [ ]:
# Install dependencies
!pip install --upgrade pip -qq
!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip install -U numpy -qq
!pip install .. -qq
!pip install -U pandas==1.5.3 -qq # Google colab requires pandas downgrade
!pip install git+https://github.com/danielgomezmarin/rhythmtoolbox -qq

In [ ]:
import torch
import yaml
from model_utils import get_model_name, load_config, save_model, get_loss_fn
from rhythmic_relationships.data import PartDatasetSequential
from rhythmic_relationships.model import TransformerDecoder
from rhythmic_relationships.train import train_transformer_decoder
from torch.utils.data import DataLoader, random_split

DEVICE = torch.device('cuda')
CONFIG_FILEPATH = "transformer_decoder_config.yml"


config = load_config(CONFIG_FILEPATH)

# Overwrite hyperparameters
config['dataset'] = {
  "dataset_name": "lmdc_500_2bar_4res",
  "part": "Melody",
  "representation": "onset_roll",
  "context_len": 16,
}
config['model'] = {
  'vocab_size': 130,
  'n_embed': 144,
  'n_head': 4,
  'n_layer': 4,
  'dropout': 0.3,
}
config['wandb'] = False
config['eval_iters'] = 10
config['num_epochs'] = 3
config['batch_size'] = 128
config['lr'] = 3e-4


print(yaml.dump(config))

torch.manual_seed(13)

dataset = PartDatasetSequential(**config["dataset"], datasets_dir='../../datasets')
splits = [0.6, 0.3, 0.1]
train_data, val_data, test_data = random_split(dataset, splits)
print(f"{splits=}: {len(train_data)}, {len(val_data)}, {len(test_data)}")

train_loader = DataLoader(train_data, batch_size=config["batch_size"], shuffle=True)
val_loader = DataLoader(val_data, batch_size=config["batch_size"], shuffle=True)

model_name = get_model_name()
print(f"{model_name=}")

config["model"]["context_len"] = config["dataset"]["context_len"]
model = TransformerDecoder(**config["model"]).to(DEVICE)
# model = BigramDecoder(vocab_size=vocab_size).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])
loss_fn = get_loss_fn(config)

evaluation = train_transformer_decoder(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    config=config,
    device=DEVICE,
    model_name=model_name,
)

stats = {
    "train_loss": evaluation["train_loss"],
    "val_loss": evaluation["val_loss"],
    "n_params": sum(p.nelement() for p in model.parameters()),
}
print(stats)

save_model(model, config, model_name, stats, bento=False)